# Preprosess Products Data

In [1]:
import pandas as pd 
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [2]:
products = pd.read_csv('producto_tabla.csv')

# Extract short name for the NombreProducto col
products['short_name'] = products.NombreProducto.str.extract('^(\D*)', expand=False)
# Extract brand for NombreProducto col
products['brand'] = products.NombreProducto.str.extract('^.+\s(\D+) \d+$', expand=False)
# Extract weight from NombreProducto col
w = products.NombreProducto.str.extract('(\d+)(Kg|g)', expand=True)
products['weight'] = w[0].astype('float')*w[1].map({'Kg':1000, 'g':1})
# Extract the # of pieces from NombreProducto col
products['pieces'] =  products.NombreProducto.str.extract('(\d+)p ', expand=False).astype('float')
# Shorten the product name using spanish stopwords
products['short_name_processed'] = (products['short_name'].map(lambda x: " "
                                    .join([i for i in x.lower()
                                    .split() if i not in stopwords.words("spanish")])))

# Keep shortening the product name 
stemmer = SnowballStemmer("spanish")
products['short_name_processed'] = (products['short_name_processed'].map(lambda x: " "
                                    .join([stemmer.stem(i) for i in x.lower().split()])))

# Drop cols
products = products.drop(columns = ['NombreProducto','short_name'],axis = 1)
# Rename cols
products = products.rename(columns = {'short_name_processed':'Producto_name'})
products = products.rename(columns = {'Producto_ID':'Producto_ID'})
# Reorder cols
products = products[['Producto_ID','Producto_name','brand','weight','pieces']]
products[:10]

# Add a col called "weight_per_piece"
products['weight_per_piece'] = products['weight']/products['pieces']

# Preprosess Clients Data 

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Visualize the data first

In [5]:
clients = pd.read_csv('cliente_tabla.csv')
clients.shape

(935362, 2)

In [6]:
clients.head()

,Cliente_ID,NombreCliente
0,0,SIN NOMBRE
1,1,OXXO XINANTECATL
2,2,SIN NOMBRE
3,3,EL MORENO
4,4,SDN SER DE ALIM CUERPO SA CIA DE INT


In [7]:
# Remove duplicate ids
duplicate_ids = clients.duplicated(subset = 'Cliente_ID')    
clients = clients[duplicate_ids == False]    
clients.shape

(930500, 2)

In [8]:
# number of unique clients names
print(str(len(clients['NombreCliente'].unique())))

307009


Note: Different ids can have the same clients name

As we can see, there are a lot of unique clients names. Let's try to find some similarities and features between them and group them by those similarities.

In [9]:
# frequencies of clients names

# count how many times the same name showed up using groupby
common_client_names = pd.DataFrame({'Frequency': clients.groupby(['NombreCliente'])['NombreCliente']
                                    .count()}).reset_index()

common_client_names['Percent'] = 100.0 * common_client_names['Frequency'] / common_client_names['Frequency'].sum()
    
common_client_names = common_client_names.sort_values(by = 'Frequency', ascending = False)
common_client_names[:10]

,NombreCliente,Frequency,Percent
227063,NO IDENTIFICADO,281670,30.270822
178995,LUPITA,4863,0.522622
203200,MARY,3016,0.324127
163963,LA PASADITA,2426,0.260720
165834,LA VENTANITA,2267,0.243632
162357,LA GUADALUPANA,1299,0.139602
265099,ROSY,1245,0.133799
20891,ALEX,1242,0.133477
120794,GABY,1238,0.133047
161465,LA ESCONDIDA,1216,0.130682


Except NO IDENTIFICADO (Non-identified), the names occur the most are colloquial names of individual clients like Mary, Rosy, Alex, or Gaby and the others are companies with stopwords in Spanish like LA and EL. 

In [10]:
# Import libraries for handling text data

import re
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

stemmer = SnowballStemmer('spanish')

# Create copy of NombreCliente named Client_Type
# Convert to lowercase
clients['Client_Type'] = clients['NombreCliente'].str.lower()
clients.head()

,Cliente_ID,NombreCliente,Client_Type
0,0,SIN NOMBRE,sin nombre
1,1,OXXO XINANTECATL,oxxo xinantecatl
2,2,SIN NOMBRE,sin nombre
3,3,EL MORENO,el moreno
4,4,SDN SER DE ALIM CUERPO SA CIA DE INT,sdn ser de alim cuerpo sa cia de int


In [11]:
# Split words in Client_Type variable
nombre_split = clients['Client_Type'].str.split()
    
# Remove empty spaces from names
nombre_no_spaces = [" ".join(x) for x in nombre_split]  

# Remove non-letters and replace numbers with 'NUM'
nombre_no_spaces = [re.sub('[^A-Za-z0-9]+', ' ', x) for x in nombre_no_spaces]   
nombre_no_spaces = [re.sub('[0-9]+', 'NUM', x) for x in nombre_no_spaces]

nombre_no_spaces[0:10]

['sin nombre',
 'oxxo xinantecatl',
 'sin nombre',
 'el moreno',
 'sdn ser de alim cuerpo sa cia de int',
 'la vaquita',
 'lupita',
 'i m el guero',
 'mini super los lupes',
 'super kompras micro colon']

In [11]:
# Stem and tokenize words 

nombre_stem = [stemmer.stem(x) for x in nombre_no_spaces]
nombre_tokenized = [word_tokenize(x) for x in nombre_stem]
nombre_tokenized[0:10]

[['sin', 'nombr'],
 ['oxxo', 'xinantecatl'],
 ['sin', 'nombr'],
 ['el', 'moren'],
 ['sdn', 'ser', 'de', 'alim', 'cuerpo', 'sa', 'cia', 'de', 'int'],
 ['la', 'vaquit'],
 ['lupit'],
 ['i', 'm', 'el', 'guer'],
 ['mini', 'super', 'los', 'lup'],
 ['super', 'kompras', 'micro', 'colon']]

In [12]:
# Remove stop words

nombre_tokenized = [word_tokenize(x) for x in clients.ix[:, 'Client_Type']]
stops = set(stopwords.words("spanish"))
nombre_cleaned = [[w for w in i if not w in stops] for i in nombre_tokenized] 
nombre_cleaned = [" ".join(x) for x in nombre_cleaned]  
nombre_cleaned[0:30]

['nombre',
 'oxxo xinantecatl',
 'nombre',
 'moreno',
 'sdn ser alim cuerpo sa cia int',
 'vaquita',
 'lupita',
 'i m guero',
 'mini super lupes',
 'super kompras micro colon',
 'lonja mercantil',
 'farmacia nicolas san juan',
 'papeleria catala',
 'elena',
 'casa trino',
 'fma035947 bimbo sa cv',
 'joys',
 'marco',
 'lupes ii',
 'tiendita',
 'fma026712 tecnoautomotriz atlacomulco s',
 'abarrotes ivan',
 'garnica',
 'fma026709 bucirus bladas mexico sa',
 'fma026248 proveedora norte',
 'carrosita',
 'bodega comercial mexicana toluca',
 'marquez',
 'fma033097 automotriz mexicana s c',
 'jose aguilar catalan']

In the 'company' category, we found that there are some obvious subcategories. For example, OXXO (convenience store), abarrotes (grocery store), and super (super market) are 3 subcategories.

Thus, we perform a client classification as follow.

Cited from https://www.kaggle.com/abbysobh/grupo-bimbo-inventory-demand/classifying-client-type-using-client-names

In [13]:
def create_client_features(clients):
    """ Creates new variable 'Client_Type' by categorizing NombreCliente
    """
    
    # Create new feature
    clients = clients.copy()
    clients['Client_Type'] = clients.ix[:, 'NombreCliente']    
    
    # Convert to all UPPER-CASE
    clients.ix[:, 'Client_Type'] = clients.ix[:, 'Client_Type'].str.upper()
    
    # Known Large Company / Special Group Types
    clients.ix[:, 'Client_Type'] = clients.ix[:, 'Client_Type'].str.replace('.*REMISION.*','Consignment')
    clients.ix[:, 'Client_Type'] = clients.ix[:, 'Client_Type'].replace(['.*WAL MART.*','.*SAMS CLUB.*'],'Walmart', regex=True)
    clients.ix[:, 'Client_Type'] = clients.ix[:, 'Client_Type'].str.replace('.*OXXO.*','Oxxo Store')
    clients.ix[:, 'Client_Type'] = clients.ix[:, 'Client_Type'].str.replace('.*CONASUPO.*','Govt Store')
    clients.ix[:, 'Client_Type'] = clients.ix[:, 'Client_Type'].str.replace('.*BIMBO.*','Bimbo Store')
    
    # Term search for assortment of words picked from looking at their frequencies
    clients.ix[:, 'Client_Type'] = clients.ix[:, 'Client_Type'].replace(['.*COLEG.*','.*UNIV.*','.*ESCU.*','.*INSTI.*',\
                                                        '.*PREPAR.*'],'School', regex=True)
    clients.ix[:, 'Client_Type'] = clients.ix[:, 'Client_Type'].str.replace('.*PUESTO.*','Post')
    clients.ix[:, 'Client_Type'] = clients.ix[:, 'Client_Type'].replace(['.*FARMA.*','.*HOSPITAL.*','.*CLINI.*'],'Hospital/Pharmacy', regex=True)
    clients.ix[:, 'Client_Type'] = clients.ix[:, 'Client_Type'].replace(['.*CAFE.*','.*CREMERIA.*','.*DULCERIA.*',\
                                                        '.*REST.*','.*BURGER.*','.*TACO.*', '.*TORTA.*',\
                                                        '.*TAQUER.*','.*HOT DOG.*',\
                                                        '.*COMEDOR.*', '.*ERIA.*','.*BURGU.*'],'Eatery', regex=True)
    clients.ix[:, 'Client_Type'] = clients.ix[:, 'Client_Type'].str.replace('.*SUPER.*','Supermarket')
    clients.ix[:, 'Client_Type'] = clients.ix[:, 'Client_Type'].replace(['.*COMERCIAL.*','.*BODEGA.*','.*DEPOSITO.*',\
                                                            '.*ABARROTES.*','.*MERCADO.*','.*CAMBIO.*',\
                                                        '.*MARKET.*','.*MART .*','.*MINI .*',\
                                                        '.*PLAZA.*','.*MISC.*','.*ELEVEN.*','.*EXP.*',\
                                                         '.*SNACK.*', '.*PAPELERIA.*', '.*CARNICERIA.*',\
                                                         '.*LOCAL.*','.*COMODIN.*','.*PROVIDENCIA.*'
                                                        ],'General Market/Mart'\
                                                       , regex=True)                                                   
    clients.ix[:, 'Client_Type'] = clients.ix[:, 'Client_Type'].replace(['.*VERDU.*','.*FRUT.*'],'Fresh Market', regex=True)
    clients.ix[:, 'Client_Type'] = clients.ix[:, 'Client_Type'].replace(['.*HOTEL.*','.*MOTEL.*'],'Hotel', regex=True)    
 
    # Filter participles
    clients.ix[:, 'Client_Type'] = clients.ix[:, 'Client_Type'].replace([
            '.*LA .*','.*EL .*','.*DE .*','.*LOS .*','.*DEL .*','.*Y .*', '.*SAN .*', '.*SANTA .*',\
            '.*AG .*','.*LAS .*','.*MI .*','.*MA .*', '.*II.*', '.*[0-9]+.*'\
                ],'Small Franchise', regex=True)
               
    # Everything else bucketed into 'Individual'
    def filter_remaining(clients):
        def function_word(data):
            # Avoid the single-words created so far by checking for upper-case
            if (data.isupper()) and (data != "NO IDENTIFICADO"): 
                return 'Individual'
            else:
                return data
        clients.ix[:, 'Client_Type'] = clients.ix[:, 'Client_Type'].map(function_word)
    filter_remaining(clients)
    
    # Return data
    return clients

In [14]:
clients = create_client_features(clients)
clients.head()

,Cliente_ID,NombreCliente,Client_Type
0,0,SIN NOMBRE,Individual
1,1,OXXO XINANTECATL,Oxxo Store
2,2,SIN NOMBRE,Individual
3,3,EL MORENO,Small Franchise
4,4,SDN SER DE ALIM CUERPO SA CIA DE INT,Small Franchise


Now we calculate our new categories' frequencies

In [15]:
len(clients['Client_Type'].unique())

16

In [16]:
common_client_types = pd.DataFrame({'Frequency': clients.groupby(['Client_Type'])['Client_Type']
                                    .count()}).reset_index()

common_client_types['Percent'] = 100.0 * common_client_types['Frequency'] / common_client_types['Frequency'].sum()
common_client_types = common_client_types.sort_values(by = 'Frequency', ascending = False)
common_client_types

,Client_Type,Frequency,Percent
8,Individual,351908,37.819237
9,NO IDENTIFICADO,281670,30.270822
13,Small Franchise,158357,17.018485
4,General Market/Mart,65516,7.040946
2,Eatery,30277,3.253842
14,Supermarket,15911,1.709941
10,Oxxo Store,9276,0.996883
6,Hospital/Pharmacy,5693,0.611822
12,School,5562,0.597743
11,Post,2658,0.285653


# Preprosessing training/test data

In [12]:
train = pd.read_csv('train.csv')

Because Venta_uni_hoy, Venta_hoy, Dev_uni_proxima, and Dev_proxima are highly correlated with Demanda_uni_equil (Adjusted Demand), and they are used in the calculation to derive Adjusted Demand, we choose to remove them from our training dataset.

In [13]:
train = train.drop(columns = ['Venta_uni_hoy','Venta_hoy','Dev_uni_proxima','Dev_proxima'], axis=1)

In [14]:
# Attach the product_name to our training set using Merge function 
train = train.merge(products, on ='Producto_ID',how='inner')

# Attach the clients_type to our training set using Merge function
train = train.merge(clients, on = 'Cliente_ID', how = 'left')

In [20]:
test = pd.read_csv('test.csv')
# Attach the product_name to our test set using Merge function 
test = test.merge(products, on ='Producto_ID',how='inner')

# Attach the clients_type to our test set using Merge function
test = test.merge(clients, on = 'Cliente_ID', how = 'left')

# Add time series lag variables to data

In [21]:

def add_time_series_features(train, test):
    """ Takes train and test data as inputs. 
        Adds lagged adjusted demand variables to the data periods t-2, t-3, t-4, and t-5.
        Returns modified data. 
    """
    # Add lagged time series variables to train/test
    # Groups by (Cliente_ID, Producto_ID)
    for i in range(1, 6):
        # Add tminus to train
        columns = ['Semana', 'Cliente_ID', 'Producto_ID', 'Demanda_uni_equil']       
        train_tminus = train.ix[:, columns]
        train_tminus = train_tminus.rename(columns = {'Demanda_uni_equil': 'Demanda_uni_equil_tminus' + str(i)})
        train_tminus['Semana'] = train_tminus['Semana'] + i
        columns.remove('Demanda_uni_equil')
        train_tminus = pd.DataFrame({'Demanda_uni_equil_tminus' + str(i) : train_tminus.groupby(columns)['Demanda_uni_equil_tminus' + str(i)].mean()}).reset_index()         
        
        train = train.merge(train_tminus, how = 'left', on = ['Semana', 'Cliente_ID', 'Producto_ID'])   
        train['Demanda_uni_equil_tminus' + str(i)] = train['Demanda_uni_equil_tminus' + str(i)].astype(np.float16)
                
        # Add tminus for Semana == 10
        columns = ['Semana', 'Cliente_ID', 'Producto_ID', 'Demanda_uni_equil_tminus' + str(i)]
        test = test.merge(train_tminus.ix[train_tminus['Semana'] == 10, columns], how = 'left', on = ['Semana', 'Cliente_ID', 'Producto_ID'])
        test['Demanda_uni_equil_tminus' + str(i)] = test['Demanda_uni_equil_tminus' + str(i)].astype(np.float16)     
    
        # Add tminus for Semana == 11
        if(i != 1):
            test = test.merge(train_tminus.ix[train_tminus['Semana'] == 11, columns], how = 'left', on = ['Semana', 'Cliente_ID', 'Producto_ID']) 
            test = test.rename(columns = {'Demanda_uni_equil_tminus' + str(i) + '_x': 'Demanda_uni_equil_tminus' + str(i)})
            test.ix[test['Semana'] == 11, 'Demanda_uni_equil_tminus' + str(i)] = test.ix[test['Semana'] == 11, 'Demanda_uni_equil_tminus' + str(i) + '_y']
            test = test.drop('Demanda_uni_equil_tminus' + str(i) + '_y', axis = 1)
            test['Demanda_uni_equil_tminus' + str(i)] = test['Demanda_uni_equil_tminus' + str(i)].astype(np.float16)
        
        # Replace null values with zeros
        train.ix[train['Demanda_uni_equil_tminus' + str(i)].isnull(), 'Demanda_uni_equil_tminus' + str(i)] = 0
        test.ix[test['Demanda_uni_equil_tminus' + str(i)].isnull(), 'Demanda_uni_equil_tminus' + str(i)] = 0    
            
    return train, test

In [22]:
train, test = add_time_series_features(train, test)  

# Add mean weekly frequency variables for categorical id variables

Note: this step increases our model's accuracy rate by dealing with id variables.

In [23]:
def add_mean_freq_by_id(train, test):
    """ Takes train and test data as inputs. 
        Adds mean of frequencies of different id features by week (semana).
        Returns train and test data.
    """
    columns =  ['Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Cliente_ID', 'Producto_ID', 'Producto_name', 'Client_Type'] 
    for column in columns:
        
        # Create mean of weekly id counts
        train_counts = pd.DataFrame({column + '_count' : train[[column, 'Semana']].groupby([column, 'Semana']).size()}).reset_index()
        test_counts = pd.DataFrame({column + '_count' : test[[column, 'Semana']].groupby([column, 'Semana']).size()}).reset_index()
        counts = train_counts.append(test_counts)      
        counts = pd.DataFrame({column + '_count' : counts.groupby([column])[column + '_count'].mean()}).reset_index()
        counts[column + '_count'] = counts[column + '_count'].astype(np.float32) 

        # Merge with train and test data
        train = train.merge(counts, how = 'left', on = column)    
        test = test.merge(counts, how = 'left', on = column)  
        
    return train, test

In [24]:
train, test = add_mean_freq_by_id(train, test)

# Encode categorical variables¶

In [27]:
from sklearn import preprocessing

def encode_labels(train, test, threshold): 
    """ Converts categorical features to integers in train and test data.
        Groups values in categorical features appearing less than threshold into a separate category.
        Returns train and test data.
    """
    
    # Identify columns for label encoding
    columns = list(train.columns[train.dtypes == 'object'])
       
    # Return data if no columns identified   
    if(len(columns) == 0):
        return train, test
       
    # Transform columns
    for column in columns: 
        
        # Filter data
        classes = train[column].unique()
        counts = train[column].value_counts()
        counts_classes = counts.index[counts <= threshold]
        
        # Set classes under threshold to 'identific'
        train.ix[train[column].isin(counts_classes), column] = 'identific'        
        test.ix[test[column].isin(counts_classes), column] = 'identific'          
        
        # Classes in test not in train sent to 'identific'
        test.ix[test[column].isin(classes) == False, column] = 'identific'
        
        # Perform label encoding
        le = preprocessing.LabelEncoder()
        le.fit(train[column])
        train[column] = le.transform(train[column]).astype(np.uint32)
        test[column] = le.transform(test[column]).astype(np.uint32)
        
    return train, test

In [28]:
train, test = encode_labels(train, test, threshold = 5)

# Remove date before Week 6

In [29]:
train = train[train['Semana'] > 5]  
train = train.reset_index(drop=True)

# Reorder columns in data¶

In [30]:
columns = [
            'Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Cliente_ID', 'Producto_ID'
            , 'Client_Type', 'Producto_name' 
            , 'weight', 'pieces', 'weight_per_piece'
            , 'Demanda_uni_equil', 'Demanda_uni_equil_tminus1', 'Demanda_uni_equil_tminus2'
            , 'Demanda_uni_equil_tminus3', 'Demanda_uni_equil_tminus4', 'Demanda_uni_equil_tminus5'
            , 'Agencia_ID_count', 'Canal_ID_count', 'Ruta_SAK_count', 'Cliente_ID_count'
            , 'Producto_ID_count', 'Client_Type_count'
          ]

train = train.reindex(columns = columns)
columns.remove('Demanda_uni_equil')
test = test.reindex(columns = columns)

# Write data to CSV files

In [31]:
train.to_csv("train_modified.csv", index = False, header = True)
test.to_csv("test_modified.csv", index = False, header = True)